# 07 - Hyperparameter Sweeps with Weights & Biases

Learn how to optimize model hyperparameters using W&B sweeps.

## Table of Contents
1. [Overview](#1.-Overview)
2. [W&B Setup](#2.-W&B-Setup)
3. [Sweep Configuration](#3.-Sweep-Configuration)
4. [Creating & Launching Sweeps](#4.-Creating-&-Launching-Sweeps)
5. [Monitoring Sweep Progress](#5.-Monitoring-Sweep-Progress)
6. [Analyzing Sweep Results](#6.-Analyzing-Sweep-Results)
7. [Extracting Best Configuration](#7.-Extracting-Best-Configuration)
8. [Sweep Visualization](#8.-Sweep-Visualization)
9. [Post-Sweep Workflow](#9.-Post-Sweep-Workflow)

---

## 1. Overview

Weights & Biases (W&B) sweeps automate hyperparameter optimization:

- **Grid Search**: Exhaustive search over specified values
- **Random Search**: Random sampling from distributions
- **Bayesian Optimization**: Intelligent search using prior results

### Sweep Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                     W&B Sweep Controller                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   ┌─────────────┐   ┌─────────────┐   ┌─────────────┐          │
│   │   Agent 1   │   │   Agent 2   │   │   Agent N   │          │
│   │  (GPU 0)    │   │  (GPU 1)    │   │  (GPU N)    │          │
│   └──────┬──────┘   └──────┬──────┘   └──────┬──────┘          │
│          │                 │                 │                  │
│          ▼                 ▼                 ▼                  │
│   ┌─────────────────────────────────────────────────┐          │
│   │              Training Runs                       │          │
│   │    (Each with different hyperparameters)        │          │
│   └─────────────────────────────────────────────────┘          │
│                          │                                      │
│                          ▼                                      │
│   ┌─────────────────────────────────────────────────┐          │
│   │           Results Dashboard & Analysis           │          │
│   └─────────────────────────────────────────────────┘          │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# Environment Setup
# ============================================================

import sys
from pathlib import Path
import json
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Project root
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

print("="*60)
print("G-CODE FINGERPRINTING - HYPERPARAMETER SWEEPS")
print("="*60)
print(f"Project Root: {project_root}")

## 2. W&B Setup

Configure Weights & Biases for sweep management.

In [ ]:
# Check W&B installation and login status
try:
    import wandb
    print(f"W&B version: {wandb.__version__}")
    
    # Check if logged in
    try:
        api = wandb.Api()
        print(f"\n✓ Logged in as: {api.viewer()['username']}")
    except Exception:
        print("\n⚠ Not logged in. Run: wandb login")
        
except ImportError:
    print("W&B not installed. Install with: pip install wandb")

In [ ]:
# W&B configuration
WANDB_PROJECT = 'gcode-fingerprinting'
WANDB_ENTITY = 'your-username'  # Replace with your W&B username/team

print("\nW&B Configuration:")
print("-" * 40)
print(f"  Project: {WANDB_PROJECT}")
print(f"  Entity: {WANDB_ENTITY}")
print(f"\n  Dashboard: https://wandb.ai/{WANDB_ENTITY}/{WANDB_PROJECT}")

## 3. Sweep Configuration

Define the hyperparameter search space.

In [ ]:
# List available sweep configs
config_dir = project_root / 'configs'
sweep_configs = list(config_dir.glob('sweep*.yaml'))

print("\nAvailable Sweep Configurations:")
print("-" * 50)

for config in sweep_configs:
    print(f"  • {config.name}")

In [ ]:
# Load and examine a sweep config
config_path = project_root / 'configs' / 'sweep_comprehensive.yaml'

if config_path.exists():
    with open(config_path, 'r') as f:
        sweep_config = yaml.safe_load(f)
    
    print("\nSweep Configuration Summary:")
    print("="*50)
    print(f"  Name: {sweep_config.get('name', 'unnamed')}")
    print(f"  Project: {sweep_config.get('project', 'N/A')}")
    print(f"  Method: {sweep_config.get('method', 'N/A')}")
    
    metric = sweep_config.get('metric', {})
    print(f"  Metric: {metric.get('name', 'N/A')} ({metric.get('goal', 'maximize')})")
    print(f"  Run Cap: {sweep_config.get('run_cap', 'unlimited')}")
    
    print(f"\n  Parameters ({len(sweep_config.get('parameters', {}))})")
    for param, spec in list(sweep_config.get('parameters', {}).items())[:10]:
        if 'value' in spec:
            print(f"    • {param}: {spec['value']} (fixed)")
        elif 'values' in spec:
            print(f"    • {param}: {spec['values']} (categorical)")
        elif 'distribution' in spec:
            print(f"    • {param}: {spec['distribution']} [{spec.get('min', 'N/A')}, {spec.get('max', 'N/A')}]")
else:
    print("\nNo sweep config found. Creating example...")

In [ ]:
# Example: Create a minimal sweep config programmatically
example_sweep_config = {
    'name': 'example_sweep',
    'project': WANDB_PROJECT,
    'method': 'bayes',  # Options: 'grid', 'random', 'bayes'
    'metric': {
        'name': 'val/composite_acc',
        'goal': 'maximize'
    },
    'run_cap': 50,
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 25,
        's': 2
    },
    'parameters': {
        # Fixed parameters
        'data-dir': {'value': 'outputs/processed_v2'},
        'vocab-path': {'value': 'data/gcode_vocab_v2.json'},
        'max-epochs': {'value': 100},
        
        # Architecture search
        'hidden_dim': {'values': [128, 192, 256]},
        'num_heads': {'values': [4, 8]},
        'num_layers': {'values': [2, 4, 6]},
        
        # Training hyperparameters
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {'values': [16, 32, 64]},
        'dropout': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.4
        },
        'weight_decay': {
            'distribution': 'log_uniform_values',
            'min': 0.01,
            'max': 0.1
        },
    }
}

print("\nExample Sweep Config:")
print("-" * 40)
print(yaml.dump(example_sweep_config, default_flow_style=False)[:1000])

## 4. Creating & Launching Sweeps

Two ways to launch sweeps: command line or programmatically.

In [ ]:
# Command-line method (recommended for production)
print("\nCommand-Line Sweep Launch:")
print("="*50)

print("""
# Step 1: Create the sweep
wandb sweep configs/sweep_comprehensive.yaml

# This will output a sweep ID like:
# wandb: Created sweep with ID: abc123xy
# wandb: View sweep at: https://wandb.ai/your-entity/gcode-fingerprinting/sweeps/abc123xy

# Step 2: Launch agent(s)
wandb agent your-entity/gcode-fingerprinting/abc123xy

# Run multiple agents in parallel (on different terminals/GPUs):
CUDA_VISIBLE_DEVICES=0 wandb agent your-entity/gcode-fingerprinting/abc123xy &
CUDA_VISIBLE_DEVICES=1 wandb agent your-entity/gcode-fingerprinting/abc123xy &
CUDA_VISIBLE_DEVICES=2 wandb agent your-entity/gcode-fingerprinting/abc123xy &
""")

In [ ]:
# Programmatic method (for notebooks/automation)
print("\nProgrammatic Sweep Launch:")
print("="*50)

print("""
import wandb

# Initialize sweep
sweep_id = wandb.sweep(
    sweep=sweep_config,
    project='gcode-fingerprinting',
    entity='your-username'
)

print(f'Created sweep: {sweep_id}')

# Define training function
def train():
    with wandb.init() as run:
        config = wandb.config
        
        # Build model with config parameters
        model = build_model(
            hidden_dim=config.hidden_dim,
            num_heads=config.num_heads,
            num_layers=config.num_layers,
            dropout=config.dropout,
        )
        
        # Train and log metrics
        for epoch in range(config.max_epochs):
            train_loss, val_acc = train_epoch(model, ...)
            wandb.log({'train/loss': train_loss, 'val/accuracy': val_acc})

# Launch agent
wandb.agent(sweep_id, function=train, count=50)
""")

## 5. Monitoring Sweep Progress

Track sweep execution in real-time.

In [ ]:
# Check sweep status via API
def get_sweep_status(sweep_id, entity, project):
    """Get current sweep status."""
    try:
        api = wandb.Api()
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        
        runs = list(sweep.runs)
        
        return {
            'name': sweep.name,
            'state': sweep.state,
            'total_runs': len(runs),
            'running': sum(1 for r in runs if r.state == 'running'),
            'finished': sum(1 for r in runs if r.state == 'finished'),
            'failed': sum(1 for r in runs if r.state == 'failed'),
            'best_run': sweep.best_run().name if sweep.best_run() else None,
        }
    except Exception as e:
        return {'error': str(e)}

# Example usage (replace with actual sweep ID)
# status = get_sweep_status('abc123xy', WANDB_ENTITY, WANDB_PROJECT)
# print(json.dumps(status, indent=2))

print("\nTo check sweep status:")
print("  status = get_sweep_status('SWEEP_ID', ENTITY, PROJECT)")

In [ ]:
# Visualize sweep progress over time
print("\nSweep Progress Visualization:")
print("-" * 50)

# Simulated sweep progress data
np.random.seed(42)
n_runs = 50

# Simulate Bayesian optimization finding better hyperparameters over time
base_acc = 0.60
improvement = np.cumsum(np.random.exponential(0.01, n_runs))
noise = np.random.randn(n_runs) * 0.03
val_accs = base_acc + improvement * 0.15 + noise
val_accs = np.clip(val_accs, 0.5, 0.95)

# Running best
running_best = np.maximum.accumulate(val_accs)

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

ax.scatter(range(n_runs), val_accs, alpha=0.6, s=50, label='Individual Runs')
ax.plot(running_best, 'r-', linewidth=2, label='Best So Far')
ax.axhline(y=running_best[-1], color='g', linestyle='--', alpha=0.5, label=f'Final Best: {running_best[-1]:.3f}')

ax.set_xlabel('Run Number')
ax.set_ylabel('Validation Accuracy')
ax.set_title('Bayesian Optimization Sweep Progress')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Analyzing Sweep Results

Extract and analyze completed sweep data.

In [ ]:
# Fetch sweep runs via API
def get_sweep_runs(sweep_id, entity, project):
    """Fetch all runs from a sweep."""
    try:
        api = wandb.Api()
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        
        runs_data = []
        for run in sweep.runs:
            if run.state == 'finished':
                config = {k: v for k, v in run.config.items() if not k.startswith('_')}
                summary = dict(run.summary)
                
                runs_data.append({
                    'run_id': run.id,
                    'name': run.name,
                    **config,
                    **{f'metric_{k}': v for k, v in summary.items() if isinstance(v, (int, float))}
                })
        
        return pd.DataFrame(runs_data)
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()

print("\nTo analyze sweep results:")
print("  df = get_sweep_runs('SWEEP_ID', ENTITY, PROJECT)")
print("  print(df.describe())")

In [ ]:
# Simulated sweep results for analysis
np.random.seed(42)
n_runs = 100

# Generate simulated sweep data
sweep_results = pd.DataFrame({
    'hidden_dim': np.random.choice([128, 192, 256, 320], n_runs),
    'num_heads': np.random.choice([4, 6, 8], n_runs),
    'num_layers': np.random.choice([2, 4, 6], n_runs),
    'learning_rate': 10 ** np.random.uniform(-5, -3, n_runs),
    'batch_size': np.random.choice([16, 32, 64], n_runs),
    'dropout': np.random.uniform(0.1, 0.4, n_runs),
    'weight_decay': 10 ** np.random.uniform(-2, -1, n_runs),
})

# Simulate validation accuracy (higher for optimal hyperparameters)
optimal_hidden = 256
optimal_lr = 5e-4

base_acc = 0.65
hidden_bonus = 0.05 * (1 - np.abs(sweep_results['hidden_dim'] - optimal_hidden) / 200)
lr_bonus = 0.05 * (1 - np.abs(np.log10(sweep_results['learning_rate']) - np.log10(optimal_lr)) / 2)
noise = np.random.randn(n_runs) * 0.02

sweep_results['val_accuracy'] = np.clip(base_acc + hidden_bonus + lr_bonus + noise, 0.5, 0.95)
sweep_results['val_loss'] = 0.5 * (1 - sweep_results['val_accuracy']) + np.random.randn(n_runs) * 0.05

print("\nSimulated Sweep Results Summary:")
print("="*50)
print(sweep_results.describe())

## 7. Extracting Best Configuration

Find the optimal hyperparameters from completed sweep.

In [ ]:
# Find best run
best_idx = sweep_results['val_accuracy'].idxmax()
best_run = sweep_results.loc[best_idx]

print("\nBest Hyperparameters:")
print("="*50)
print(f"  Validation Accuracy: {best_run['val_accuracy']:.4f}")
print(f"  Validation Loss: {best_run['val_loss']:.4f}")
print("\n  Configuration:")
print(f"    hidden_dim: {int(best_run['hidden_dim'])}")
print(f"    num_heads: {int(best_run['num_heads'])}")
print(f"    num_layers: {int(best_run['num_layers'])}")
print(f"    learning_rate: {best_run['learning_rate']:.2e}")
print(f"    batch_size: {int(best_run['batch_size'])}")
print(f"    dropout: {best_run['dropout']:.3f}")
print(f"    weight_decay: {best_run['weight_decay']:.4f}")

In [ ]:
# Generate training command with best hyperparameters
print("\nTraining Command with Best Hyperparameters:")
print("="*50)

command = f"""PYTHONPATH=src .venv/bin/python scripts/train_multihead.py \\
    --data-dir outputs/processed_v2 \\
    --vocab-path data/gcode_vocab_v2.json \\
    --output-dir outputs/final_model \\
    --hidden-dim {int(best_run['hidden_dim'])} \\
    --num-heads {int(best_run['num_heads'])} \\
    --num-layers {int(best_run['num_layers'])} \\
    --learning-rate {best_run['learning_rate']:.2e} \\
    --batch-size {int(best_run['batch_size'])} \\
    --dropout {best_run['dropout']:.3f} \\
    --weight-decay {best_run['weight_decay']:.4f} \\
    --max-epochs 200 \\
    --patience 20
"""

print(command)

## 8. Sweep Visualization

Visualize hyperparameter importance and interactions.

In [ ]:
# Hyperparameter importance (correlation with val_accuracy)
hp_columns = ['hidden_dim', 'num_heads', 'num_layers', 'learning_rate', 'batch_size', 'dropout', 'weight_decay']

correlations = sweep_results[hp_columns + ['val_accuracy']].corr()['val_accuracy'][:-1].sort_values(key=abs, ascending=False)

print("\nHyperparameter Importance (Correlation with Accuracy):")
print("-" * 50)

fig, ax = plt.subplots(figsize=(10, 5))

colors = ['green' if c > 0 else 'red' for c in correlations]
ax.barh(range(len(correlations)), correlations.values, color=colors, alpha=0.7)
ax.set_yticks(range(len(correlations)))
ax.set_yticklabels(correlations.index)
ax.set_xlabel('Correlation with Validation Accuracy')
ax.set_title('Hyperparameter Importance')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Parallel coordinates plot
from pandas.plotting import parallel_coordinates

# Normalize data for visualization
viz_data = sweep_results.copy()

# Create accuracy bins for coloring
viz_data['accuracy_bin'] = pd.cut(
    viz_data['val_accuracy'],
    bins=[0, 0.70, 0.75, 0.80, 1.0],
    labels=['Low', 'Medium', 'High', 'Best']
)

# Normalize numerical columns
for col in ['hidden_dim', 'num_layers', 'learning_rate', 'dropout']:
    viz_data[col] = (viz_data[col] - viz_data[col].min()) / (viz_data[col].max() - viz_data[col].min())

# Plot
fig, ax = plt.subplots(figsize=(14, 6))

parallel_coordinates(
    viz_data[['hidden_dim', 'num_layers', 'learning_rate', 'dropout', 'accuracy_bin']],
    'accuracy_bin',
    ax=ax,
    alpha=0.3
)

ax.set_title('Parallel Coordinates: Hyperparameters vs Accuracy')
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Learning rate vs Accuracy scatter
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Learning rate
axes[0].scatter(np.log10(sweep_results['learning_rate']), sweep_results['val_accuracy'], alpha=0.5, c=sweep_results['hidden_dim'], cmap='viridis')
axes[0].set_xlabel('Log10(Learning Rate)')
axes[0].set_ylabel('Validation Accuracy')
axes[0].set_title('Learning Rate vs Accuracy')

# Hidden dim
for hd in sweep_results['hidden_dim'].unique():
    mask = sweep_results['hidden_dim'] == hd
    axes[1].scatter(sweep_results.loc[mask, 'hidden_dim'] + np.random.randn(mask.sum()) * 5, 
                    sweep_results.loc[mask, 'val_accuracy'], alpha=0.5, label=f'{int(hd)}')
axes[1].set_xlabel('Hidden Dimension')
axes[1].set_ylabel('Validation Accuracy')
axes[1].set_title('Hidden Dim vs Accuracy')
axes[1].legend()

# Dropout
axes[2].scatter(sweep_results['dropout'], sweep_results['val_accuracy'], alpha=0.5, c=sweep_results['num_layers'], cmap='plasma')
axes[2].set_xlabel('Dropout')
axes[2].set_ylabel('Validation Accuracy')
axes[2].set_title('Dropout vs Accuracy (color=layers)')

plt.tight_layout()
plt.show()

## 9. Post-Sweep Workflow

Steps after sweep completion.

In [ ]:
# Post-sweep checklist
print("\nPost-Sweep Workflow:")
print("="*50)

workflow = """
1. Analyze sweep results
   - Review W&B dashboard: https://wandb.ai/YOUR_ENTITY/gcode-fingerprinting/sweeps
   - Identify best hyperparameters
   - Check for overfitting (train vs val curves)

2. Validate best configuration
   - Retrain with best hyperparameters (3-5 seeds)
   - Compute mean ± std of metrics
   - Verify reproducibility

3. Final training
   - Train for longer (more epochs, lower LR)
   - Enable all data augmentation
   - Save best checkpoint

4. Evaluation
   - Run full evaluation on test set
   - Generate confusion matrices
   - Compute per-class metrics

5. Documentation
   - Export sweep summary to CSV
   - Save best config as YAML
   - Update README with results
"""

print(workflow)

In [ ]:
# Export best config to YAML
best_config = {
    'model': {
        'hidden_dim': int(best_run['hidden_dim']),
        'num_heads': int(best_run['num_heads']),
        'num_layers': int(best_run['num_layers']),
        'dropout': float(best_run['dropout']),
    },
    'training': {
        'learning_rate': float(best_run['learning_rate']),
        'batch_size': int(best_run['batch_size']),
        'weight_decay': float(best_run['weight_decay']),
        'max_epochs': 200,
        'patience': 20,
    },
    'data': {
        'data_dir': 'outputs/processed_v2',
        'vocab_path': 'data/gcode_vocab_v2.json',
    },
    'metrics': {
        'val_accuracy': float(best_run['val_accuracy']),
        'val_loss': float(best_run['val_loss']),
    }
}

print("\nBest Configuration (YAML):")
print("-" * 40)
print(yaml.dump(best_config, default_flow_style=False))

# Save to file (uncomment to save)
# with open(project_root / 'configs' / 'best_config.yaml', 'w') as f:
#     yaml.dump(best_config, f, default_flow_style=False)

## Summary

In this notebook, you learned:

- **W&B Setup**: Configure project and authenticate
- **Sweep Configuration**: Define search space with YAML
- **Launch Methods**: Command-line and programmatic
- **Monitoring**: Track progress and early termination
- **Analysis**: Extract results and visualize importance
- **Best Config**: Export optimal hyperparameters

### Key Commands

```bash
# Create sweep
wandb sweep configs/sweep_comprehensive.yaml

# Run agent
wandb agent ENTITY/PROJECT/SWEEP_ID

# Check login
wandb login
wandb whoami
```

### Sweep Methods

| Method | Use Case |
|--------|----------|
| `grid` | Small search space, exhaustive search |
| `random` | Large space, baseline exploration |
| `bayes` | Optimal for expensive training runs |

---

**Navigation:**
← [Previous: 06_dashboard_usage](06_dashboard_usage.ipynb) |
[Next: 08_model_evaluation](08_model_evaluation.ipynb) →

**Related:** [03_training_models](03_training_models.ipynb) | [09_ablation_studies](09_ablation_studies.ipynb)